# Tools

## Data preprocessing

Load a month of monthly Neighborhood Patterns, then prepare data for IPF.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
import calendar

tqdm.pandas()

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  

pd.set_option('display.max_columns', None)

import logging
# Create a logger
logger_name = 'all_logger'
logger = logging.getLogger(logger_name)

### Load monthly Neighborhood Patterns

#### Asign file locations

In [2]:
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo

#  #### Dell desktop
save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor'   
data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'    # Dell
# data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'    # Dell

ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
##  home_panel_fname = f"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
home_panel_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\home_panel_summary_2019_2023_restated_20240119.csv.gz"  ## large change!!!
CBG_2019_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"

# home_panel_fname = r"E:\OneDrive_PSU\OneDrive - The`1

# data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Dell
# ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Lenovo
# ACS_file = r"F:\Research\Wild_fire\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv"
# CBG_2019_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"

# # 2018 desktop
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor'   
# # save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2023_home_panel'  # 2018 desktop
# # # data_dir = r'D:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'  
# data_dir = r'\\HWW\SafeGraph\Advan_2023_API\Neighborhood_Patterns'    # 2018
# # data_dir = r'D:\SafeGraph\Advan_2024_API\Neighborhood_Patterns'    
# ACS_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# # home_panel_fname = f"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
# home_panel_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\home_panel_summary_2019_2023_restated_20240119.csv.gz"  ## large change!!!
# # # https://community.deweydata.io/t/residing-device-count-in-2023-07-and-2023-08-surged-abnomorally/26675/15
# CBG_2019_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"


# save_file = False
save_file = True

stop_factor = 1

bottom_popultion_ratio = 0.1 # a CBG must have such popultion at any hours

year = '2022'
month = 7

stops_per_device = 2 # stops per device per day
stop_sampling_rate = 0.20

stop_sampling_rate_exp = 1.5

_, num_days = calendar.monthrange(int(year), month)
print(f"Days in {year}-{month:02d}:", num_days)


os.makedirs(save_dir, exist_ok=True)


month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
no_dict_colum_fname = os.path.join(save_dir, f"NP_dict_column_{year}_{month:02}.csv")
device_home_areas_fname = os.path.join(save_dir, f"NP_split_device_home_areas_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_CBG_fname     = os.path.join(save_dir, f"adjusted_dwelling_time_stop_CBG_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_natinol_fname = os.path.join(save_dir, f"adjusted_dwelling_time_stop_natinol_{year}_{month:02}.csv")
person_per_device_fname = os.path.join(save_dir, f"person_per_device_{year}_{month:02}.csv")
restored_CBG_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_CBG_{year}_{month:02}.csv")
restored_target_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_target_{year}_{month:02}.csv")
CBG_device_stop_destination_distribution_fname  = os.path.join(save_dir, f"CBG_device_stop_destination_distribution_{year}_{month:02}.csv")
CBG_device_stop_origin_distribution_fname = os.path.join(save_dir, f"CBG_device_stop_origin_distribution_{year}_{month:02}.csv")
IPF_results_fname = os.path.join(save_dir, f"IPF_origin_outgoer_hourly_{year}{month:02}.csv")
CBG_hourly_population_fname = os.path.join(save_dir, f"CBG_population_hourly_{year}{month:02}.csv")

CBG_origin_outbound_distribution_sampling_rate_fname = os.path.join(save_dir, f"CBG_origin_outbound_distribution_sampling_rate_{year}{month:02}.csv")
CBG_destination_inbound_per_device_fname = os.path.join(save_dir, f"CBG_destination_inbound_per_device_{year}{month:02}.csv")

CBG_destination_inbound_fname = os.path.join(save_dir, f"CBG_destination_inbound_{year}{month:02}.csv")
inbound_hourly_df_fname = os.path.join(save_dir, f"inbound_hourly_{year}{month:02}.csv")
CBG_agregated_inbound_fname =  os.path.join(save_dir, f"CBG_agregated_inbound_{year}{month:02}.csv")
hourly_agregated_inbound_fname =  os.path.join(save_dir, f"hourly_agregated_inbound_{year}{month:02}.csv")

 
# month_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01'
# CBG_list_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\ACS_2019_CBG_list.csv"

# sqlite_fname = r'F:\Research\Wild_fire\Neighborhood_202306.db'

  
print("Month directory:", month_dir)
print("Save_dir directory:", save_dir)
print("ACS_file:", ACS_file)
print("home_panel file name:", home_panel_fname)

print("No dict colum file name:", no_dict_colum_fname)
print("Device home areas file name:", device_home_areas_fname)
print("Device person_per_device file name:", person_per_device_fname)


print("Device adjusted_dwelling_time_stop_CBG_fname:", adjusted_dwelling_time_stop_CBG_fname)
print("Device adjusted_dwelling_time_stop_natinol_fname:", adjusted_dwelling_time_stop_natinol_fname)

print("Restored_hourly_vistor_count_CBG file name:   ", restored_CBG_hourly_vistor_count_fname)
print("Restored_hourly_vistor_count_target file name:", restored_target_hourly_vistor_count_fname)

print("CBG_device_stop_destination_distribution file name:   ", CBG_device_stop_destination_distribution_fname)
print("CBG_device_stop_origin_distribution file name:", CBG_device_stop_origin_distribution_fname)

print("IPF_results file name:", IPF_results_fname)
print("CBG hourly population file name:", CBG_hourly_population_fname)

print("CBG_origin_outbound_distribution_sampling_rate file name:", CBG_origin_outbound_distribution_sampling_rate_fname)


Days in 2022-07: 31
Month directory: F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2022\07\01
Save_dir directory: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor
ACS_file: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv
home_panel file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\home_panel_summary_2019_2023_restated_20240119.csv.gz
No dict colum file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\NP_dict_column_2022_07.csv
Device home areas file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\NP_split_device_home_areas_2022_07.csv
Device person_per_device file name: D:\On

## Load monthly neighborhood patterns

In [3]:
use_cols =  ['AREA', 
             "DEVICE_HOME_AREAS",
            "STOPS_BY_DAY", 
            "RAW_STOP_COUNTS", 
            "RAW_DEVICE_COUNTS", 
            "MEDIAN_DWELL", 
            "STOPS_BY_EACH_HOUR", 
             # 'work_behavior_device_home_areas',
             "WORK_BEHAVIOR_DEVICE_HOME_AREAS",
           
            ]
# month_df = ad_op.load_neighborhood_monthly_folder(folder=month_dir, extions=['gz'], start_str="data", use_cols=use_cols, verbose=True)
month_df = ad_op.load_neighborhood_monthly_folder(folder=month_dir, extions=['gz'], start_str="N", use_cols=use_cols, verbose=True)
# month_df = ad_op.load_neighborhood_monthly_folder(folder=month_dir, extions=['gz'], start_str="data", use_cols=use_cols, verbose=True)


# assume a device has trips_per_device (e.g., 2) stop per day !!!!!!!!!!!!!!!!!!!!!!!!!!
month_df['assumed_stops'] = month_df['RAW_DEVICE_COUNTS'].astype(int) * stops_per_device * num_days
 
month_df = month_df.sort_values("AREA")
month_df['AREA'] = month_df['AREA'].astype(str).str.zfill(12)
month_df = month_df[use_cols + ['assumed_stops']]
month_df

,AREA,DEVICE_HOME_AREAS,STOPS_BY_DAY,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,MEDIAN_DWELL,STOPS_BY_EACH_HOUR,WORK_BEHAVIOR_DEVICE_HOME_AREAS,assumed_stops
3310,010010201001,"{""010010205002"":157,""010010208021"":138,""010010...","[394,295,266,422,348,361,394,426,298,303,355,4...",11888,2991,23.600000,"[2,0,0,0,2,2,9,30,15,18,19,34,27,31,33,31,25,3...","{""010010201001"":47,""515200201003"":4,""121319506...",185442
8832,010010201002,"{""010010201002"":156,""010010205002"":62,""0100102...","[310,268,232,298,258,296,250,303,304,272,249,2...",8906,1167,216.833333,"[4,1,3,0,0,1,4,5,10,11,9,17,35,13,23,23,26,31,...","{""010010201002"":118,""010510310001"":4,""01001020...",72354
16197,010010202001,"{""010010208022"":87,""010010205002"":82,""01001020...","[208,263,241,246,226,216,199,229,213,219,325,2...",8252,1507,46.100000,"[6,0,2,0,0,0,1,6,5,4,7,6,12,17,20,17,10,21,23,...","{""010010202001"":58,""010010205002"":4,""360290168...",93434
7306,010010202002,"{""010010205002"":439,""010010206001"":260,""010010...","[1478,985,946,1461,1310,1414,1411,1409,1113,10...",41567,8602,10.283333,"[9,2,4,2,1,12,17,67,73,107,94,103,187,109,93,9...","{""010010202002"":73,""010010208021"":9,""010510309...",533324
19512,010010203001,"{""010010203001"":310,""010010205002"":147,""010010...","[696,712,740,726,655,649,662,713,672,752,754,6...",22195,2898,141.300000,"[7,4,1,2,2,4,10,22,36,34,26,35,30,35,48,49,52,...","{""010010203001"":234,""010010205002"":4,""01001020...",179676
...,...,...,...,...,...,...,...,...,...
13915,780309611001,"{""780309611001"":83,""780309602003"":35,""78030960...","[308,384,152,150,440,506,319,357,337,217,338,4...",10355,2804,30.400000,"[2,1,0,0,2,4,1,16,21,26,16,32,21,22,19,23,20,2...","{""780309611001"":33,""120970408021"":5,""780309609...",173848
5025,780309611002,"{""780309611002"":51,""780309611001"":45,""78030960...","[183,147,80,103,181,173,160,238,162,105,148,17...",4946,1293,16.650000,"[5,0,0,0,2,3,3,8,8,12,13,13,20,16,8,8,13,14,10...","{""780309611002"":25,""780309605001"":4,""780209501...",80166
6825,780309612001,"{""780309612001"":45,""780309602003"":10,""78030961...","[60,49,35,49,59,44,43,51,42,44,53,59,57,71,57,...",1595,242,116.116667,"[3,0,1,0,1,0,0,3,2,3,1,3,3,5,1,1,4,6,5,7,5,2,3...","{""780309612001"":18,""780309604004"":4}",15004
21891,780309612002,"{""780309612001"":19,""780309609001"":17,""78030961...","[132,188,73,65,245,222,172,128,113,83,120,195,...",4687,1492,22.516667,"[1,0,1,0,0,1,0,6,10,13,5,5,16,9,14,8,9,12,5,1,...","{""780309612002"":5,""120190313002"":4,""7803096030...",92504


In [4]:
print("RAW_DEVICE_COUNTS (billion):", month_df['RAW_DEVICE_COUNTS'].sum() / 1e9)
print("RAW_STOP_COUNTS (billion):", month_df['RAW_STOP_COUNTS'].sum() / 1e9)
print("assumed_stops (billion):", month_df['assumed_stops'].sum() / 1e9)


RAW_DEVICE_COUNTS (billion): 0.773563687
RAW_STOP_COUNTS (billion): 3.479020127
assumed_stops (billion): 47.960948594


In [5]:
print("MEDIAN_DWELL (minutes):", (month_df['MEDIAN_DWELL'] *  month_df['RAW_STOP_COUNTS']).sum() / month_df['RAW_STOP_COUNTS'].sum())
print("MEDIAN_DWELL (minutes):", month_df['MEDIAN_DWELL'].median())
month_df['MEDIAN_DWELL'].describe()

MEDIAN_DWELL (minutes): 33.874002488871824
MEDIAN_DWELL (minutes): 34.166666667


count    220145.000000
mean         58.795589
std          60.061205
min           1.000000
25%          11.216667
50%          34.166667
75%          90.216667
max        1413.700000
Name: MEDIAN_DWELL, dtype: float64

## Check the total of the hourly stops

Usually are 98.9% of the sum of STOPS_BY_EACH_HOUR column

In [6]:
 # month_df.iloc[:1]['WORK_BEHAVIOR_DEVICE_HOME_AREAS']

In [7]:
def _get_total_work_behavior(row):
    # print(row['WORK_BEHAVIOR_DEVICE_HOME_AREAS'])
    devices = json.loads(row['WORK_BEHAVIOR_DEVICE_HOME_AREAS'])
    # print()
    total_devices = sum(devices.values())
    return total_devices

month_df['total_work_device'] = month_df.iloc[:].apply(_get_total_work_behavior, axis=1)
# ratio = month_df['total_hourly_stop'].sum() / month_df['WORK_BEHAVIOR_DEVICE_HOME_AREAS'].sum()

# print("Ratio: total_hourly_stop / RAW_STOP_COUNTS = ", ratio)

# the total_hourly_stop is smaller than the RAW_STOP_COUNTS (98.9%)

month_df 

,AREA,DEVICE_HOME_AREAS,STOPS_BY_DAY,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,MEDIAN_DWELL,STOPS_BY_EACH_HOUR,WORK_BEHAVIOR_DEVICE_HOME_AREAS,assumed_stops,total_work_device
3310,010010201001,"{""010010205002"":157,""010010208021"":138,""010010...","[394,295,266,422,348,361,394,426,298,303,355,4...",11888,2991,23.600000,"[2,0,0,0,2,2,9,30,15,18,19,34,27,31,33,31,25,3...","{""010010201001"":47,""515200201003"":4,""121319506...",185442,91
8832,010010201002,"{""010010201002"":156,""010010205002"":62,""0100102...","[310,268,232,298,258,296,250,303,304,272,249,2...",8906,1167,216.833333,"[4,1,3,0,0,1,4,5,10,11,9,17,35,13,23,23,26,31,...","{""010010201002"":118,""010510310001"":4,""01001020...",72354,138
16197,010010202001,"{""010010208022"":87,""010010205002"":82,""01001020...","[208,263,241,246,226,216,199,229,213,219,325,2...",8252,1507,46.100000,"[6,0,2,0,0,0,1,6,5,4,7,6,12,17,20,17,10,21,23,...","{""010010202001"":58,""010010205002"":4,""360290168...",93434,70
7306,010010202002,"{""010010205002"":439,""010010206001"":260,""010010...","[1478,985,946,1461,1310,1414,1411,1409,1113,10...",41567,8602,10.283333,"[9,2,4,2,1,12,17,67,73,107,94,103,187,109,93,9...","{""010010202002"":73,""010010208021"":9,""010510309...",533324,214
19512,010010203001,"{""010010203001"":310,""010010205002"":147,""010010...","[696,712,740,726,655,649,662,713,672,752,754,6...",22195,2898,141.300000,"[7,4,1,2,2,4,10,22,36,34,26,35,30,35,48,49,52,...","{""010010203001"":234,""010010205002"":4,""01001020...",179676,286
...,...,...,...,...,...,...,...,...,...,...
13915,780309611001,"{""780309611001"":83,""780309602003"":35,""78030960...","[308,384,152,150,440,506,319,357,337,217,338,4...",10355,2804,30.400000,"[2,1,0,0,2,4,1,16,21,26,16,32,21,22,19,23,20,2...","{""780309611001"":33,""120970408021"":5,""780309609...",173848,90
5025,780309611002,"{""780309611002"":51,""780309611001"":45,""78030960...","[183,147,80,103,181,173,160,238,162,105,148,17...",4946,1293,16.650000,"[5,0,0,0,2,3,3,8,8,12,13,13,20,16,8,8,13,14,10...","{""780309611002"":25,""780309605001"":4,""780209501...",80166,53
6825,780309612001,"{""780309612001"":45,""780309602003"":10,""78030961...","[60,49,35,49,59,44,43,51,42,44,53,59,57,71,57,...",1595,242,116.116667,"[3,0,1,0,1,0,0,3,2,3,1,3,3,5,1,1,4,6,5,7,5,2,3...","{""780309612001"":18,""780309604004"":4}",15004,22
21891,780309612002,"{""780309612001"":19,""780309609001"":17,""78030961...","[132,188,73,65,245,222,172,128,113,83,120,195,...",4687,1492,22.516667,"[1,0,1,0,0,1,0,6,10,13,5,5,16,9,14,8,9,12,5,1,...","{""780309612002"":5,""120190313002"":4,""7803096030...",92504,45


In [8]:
month_df['total_work_device'].sum() / month_df['RAW_DEVICE_COUNTS'].sum()

0.030359271504945915

In [9]:
def _get_total_hourly_stop(row):
    # print(row)
    hourly_stop = json.loads(row['STOPS_BY_EACH_HOUR'])
    total_stop = sum(hourly_stop)
    return total_stop

month_df['total_hourly_stop'] = month_df.iloc[:].apply(_get_total_hourly_stop, axis=1)
ratio = month_df['total_hourly_stop'].sum() / month_df['RAW_STOP_COUNTS'].sum()

print("Ratio: total_hourly_stop / RAW_STOP_COUNTS = ", ratio)

# the total_hourly_stop is smaller than the RAW_STOP_COUNTS (98.9%)

Ratio: total_hourly_stop / RAW_STOP_COUNTS =  1.0


## Adjust the RAW_STOP_COUNTS by dwellling_time

If a stop dwelling > 60 minutes, we add a stop to the next hours. 

In [10]:
non_adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=month_df.iloc[:], adjust_dwell_time=False, clean_negative=True, stop_factor=stop_factor)
print("sum of non_adjusted_dwell_time_stop_CBG_arr (billion):", non_adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000000)
print("shape of non_adjusted_dwell_time_stop_CBG_arr:", non_adjusted_dwell_time_stop_CBG_arr.shape)

adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=month_df.iloc[:], adjust_dwell_time=True, clean_negative=True, stop_factor=stop_factor)

adjusted_dwell_time_stop_national_arr = adjusted_dwell_time_stop_CBG_arr.sum(axis=0)

month_df['adjusted_raw_stop'] = adjusted_dwell_time_stop_CBG_arr.sum(axis=1)
month_df[['adjusted_raw_stop', 'RAW_STOP_COUNTS']].sum() / 1e9

print("sum of adjusted_dwell_time_stop_CBG_arr (billion):", adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000000)
print("sum of adjusted_dwell_time_stop_national_arr (billion):", adjusted_dwell_time_stop_national_arr.sum().sum()/1000000000)
print("shape of adjusted_dwell_time_stop_CBG_arr:", adjusted_dwell_time_stop_CBG_arr.shape)

Stop factor:  1
sum of non_adjusted_dwell_time_stop_CBG_arr (billion): 3.479020127
shape of non_adjusted_dwell_time_stop_CBG_arr: (220145, 744)
Stop factor:  1
sum of adjusted_dwell_time_stop_CBG_arr (billion): 4.401263094
sum of adjusted_dwell_time_stop_national_arr (billion): 4.401263094
shape of adjusted_dwell_time_stop_CBG_arr: (220145, 744)


In [11]:
# month_df

###　Save adjusted  RAW_STOP_COUNTS 

In [12]:
# CBG level
hourly_columns = [f'hour_{h+1}' for h in range(adjusted_dwell_time_stop_CBG_arr.shape[1])]
adjusted_dwelling_time_stop_CBG_df = pd.DataFrame(adjusted_dwell_time_stop_CBG_arr, columns=hourly_columns)
adjusted_dwelling_time_stop_CBG_df['CBG'] = month_df['AREA'].astype(str).str.zfill(12).to_list()
adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df[['CBG'] + hourly_columns]  # reorder columns
adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df.sort_values('CBG')

# national level
adjusted_dwelling_time_stop_natinol_df = pd.DataFrame(adjusted_dwell_time_stop_national_arr, columns=['stop'])

adjusted_dwelling_time_stop_CBG_df

,CBG,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24,hour_25,hour_26,hour_27,hour_28,hour_29,hour_30,hour_31,hour_32,hour_33,hour_34,hour_35,hour_36,hour_37,hour_38,hour_39,hour_40,hour_41,hour_42,hour_43,hour_44,hour_45,hour_46,hour_47,hour_48,hour_49,hour_50,hour_51,hour_52,hour_53,hour_54,hour_55,hour_56,hour_57,hour_58,hour_59,hour_60,hour_61,hour_62,hour_63,hour_64,hour_65,hour_66,hour_67,hour_68,hour_69,hour_70,hour_71,hour_72,hour_73,hour_74,hour_75,hour_76,hour_77,hour_78,hour_79,hour_80,hour_81,hour_82,hour_83,hour_84,hour_85,hour_86,hour_87,hour_88,hour_89,hour_90,hour_91,hour_92,hour_93,hour_94,hour_95,hour_96,hour_97,hour_98,hour_99,hour_100,hour_101,hour_102,hour_103,hour_104,hour_105,hour_106,hour_107,hour_108,hour_109,hour_110,hour_111,hour_112,hour_113,hour_114,hour_115,hour_116,hour_117,hour_118,hour_119,hour_120,hour_121,hour_122,hour_123,hour_124,hour_125,hour_126,hour_127,hour_128,hour_129,hour_130,hour_131,hour_132,hour_133,hour_134,hour_135,hour_136,hour_137,hour_138,hour_139,hour_140,hour_141,hour_142,hour_143,hour_144,hour_145,hour_146,hour_147,hour_148,hour_149,hour_150,hour_151,hour_152,hour_153,hour_154,hour_155,hour_156,hour_157,hour_158,hour_159,hour_160,hour_161,hour_162,hour_163,hour_164,hour_165,hour_166,hour_167,hour_168,hour_169,hour_170,hour_171,hour_172,hour_173,hour_174,hour_175,hour_176,hour_177,hour_178,hour_179,hour_180,hour_181,hour_182,hour_183,hour_184,hour_185,hour_186,hour_187,hour_188,hour_189,hour_190,hour_191,hour_192,hour_193,hour_194,hour_195,hour_196,hour_197,hour_198,hour_199,hour_200,hour_201,hour_202,hour_203,hour_204,hour_205,hour_206,hour_207,hour_208,hour_209,hour_210,hour_211,hour_212,hour_213,hour_214,hour_215,hour_216,hour_217,hour_218,hour_219,hour_220,hour_221,hour_222,hour_223,hour_224,hour_225,hour_226,hour_227,hour_228,hour_229,hour_230,hour_231,hour_232,hour_233,hour_234,hour_235,hour_236,hour_237,hour_238,hour_239,hour_240,hour_241,hour_242,hour_243,hour_244,hour_245,hour_246,hour_247,hour_248,hour_249,hour_250,hour_251,hour_252,hour_253,hour_254,hour_255,hour_256,hour_257,hour_258,hour_259,hour_260,hour_261,hour_262,hour_263,hour_264,hour_265,hour_266,hour_267,hour_268,hour_269,hour_270,hour_271,hour_272,hour_273,hour_274,hour_275,hour_276,hour_277,hour_278,hour_279,hour_280,hour_281,hour_282,hour_283,hour_284,hour_285,hour_286,hour_287,hour_288,hour_289,hour_290,hour_291,hour_292,hour_293,hour_294,hour_295,hour_296,hour_297,hour_298,hour_299,hour_300,hour_301,hour_302,hour_303,hour_304,hour_305,hour_306,hour_307,hour_308,hour_309,hour_310,hour_311,hour_312,hour_313,hour_314,hour_315,hour_316,hour_317,hour_318,hour_319,hour_320,hour_321,hour_322,hour_323,hour_324,hour_325,hour_326,hour_327,hour_328,hour_329,hour_330,hour_331,hour_332,hour_333,hour_334,hour_335,hour_336,hour_337,hour_338,hour_339,hour_340,hour_341,hour_342,hour_343,hour_344,hour_345,hour_346,hour_347,hour_348,hour_349,hour_350,hour_351,hour_352,hour_353,hour_354,hour_355,hour_356,hour_357,hour_358,hour_359,hour_360,hour_361,hour_362,hour_363,hour_364,hour_365,hour_366,hour_367,hour_368,hour_369,hour_370,hour_371,hour_372,hour_373,hour_374,hour_375,hour_376,hour_377,hour_378,hour_379,hour_380,hour_381,hour_382,hour_383,hour_384,hour_385,hour_386,hour_387,hour_388,hour_389,hour_390,hour_391,hour_392,hour_393,hour_394,hour_395,hour_396,hour_397,hour_398,hour_399,hour_400,hour_401,hour_402,hour_403,hour_404,hour_405,hour_406,hour_407,hour_408,hour_409,hour_410,hour_411,hour_412,hour_413,hour_414,hour_415,hour_416,hour_417,hour_418,hour_419,hour_420,hour_421,hour_422,hour_423,hour_424,hour_425,hour_426,hour_427,hour_428,hour_429,hour_430,hour_431,hour_432,hour_433,hour_434,hour_435,hour_436,hour_437,hour_438,hour_439,hour_440,hour_441,hour_442,hour_443,hour_444,hour_445,hour_446,hour_447,hour_448,hour_449,hour_450,hour_451,hour_452,hour_453,hour_454,hour_455,hour_456

In [13]:
adjusted_dwelling_time_stop_natinol_df['stop'].sum() / 10**9

4.401263094

In [14]:
# Save CSV files
if save_file:
    adjusted_dwelling_time_stop_natinol_df.round(0).to_csv(adjusted_dwelling_time_stop_natinol_fname, index=False)
    print("Saved national hourly stop at:", adjusted_dwelling_time_stop_natinol_fname)

    adjusted_dwelling_time_stop_CBG_df.round(0).to_csv(adjusted_dwelling_time_stop_CBG_fname, index=False)
    print("Saved CBG hourly stop at:", adjusted_dwelling_time_stop_CBG_fname)

Saved national hourly stop at: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\adjusted_dwelling_time_stop_natinol_2022_07.csv
Saved CBG hourly stop at: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\adjusted_dwelling_time_stop_CBG_2022_07.csv


In [15]:
adjusted_dwelling_time_stop_natinol_df['stop'].sum() /  10**9, adjusted_dwelling_time_stop_CBG_df.iloc[:, 1:].sum().sum() /  10**9


(4.401263094, 4.401263094)

In [16]:
 adjusted_dwelling_time_stop_CBG_df

,CBG,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24,hour_25,hour_26,hour_27,hour_28,hour_29,hour_30,hour_31,hour_32,hour_33,hour_34,hour_35,hour_36,hour_37,hour_38,hour_39,hour_40,hour_41,hour_42,hour_43,hour_44,hour_45,hour_46,hour_47,hour_48,hour_49,hour_50,hour_51,hour_52,hour_53,hour_54,hour_55,hour_56,hour_57,hour_58,hour_59,hour_60,hour_61,hour_62,hour_63,hour_64,hour_65,hour_66,hour_67,hour_68,hour_69,hour_70,hour_71,hour_72,hour_73,hour_74,hour_75,hour_76,hour_77,hour_78,hour_79,hour_80,hour_81,hour_82,hour_83,hour_84,hour_85,hour_86,hour_87,hour_88,hour_89,hour_90,hour_91,hour_92,hour_93,hour_94,hour_95,hour_96,hour_97,hour_98,hour_99,hour_100,hour_101,hour_102,hour_103,hour_104,hour_105,hour_106,hour_107,hour_108,hour_109,hour_110,hour_111,hour_112,hour_113,hour_114,hour_115,hour_116,hour_117,hour_118,hour_119,hour_120,hour_121,hour_122,hour_123,hour_124,hour_125,hour_126,hour_127,hour_128,hour_129,hour_130,hour_131,hour_132,hour_133,hour_134,hour_135,hour_136,hour_137,hour_138,hour_139,hour_140,hour_141,hour_142,hour_143,hour_144,hour_145,hour_146,hour_147,hour_148,hour_149,hour_150,hour_151,hour_152,hour_153,hour_154,hour_155,hour_156,hour_157,hour_158,hour_159,hour_160,hour_161,hour_162,hour_163,hour_164,hour_165,hour_166,hour_167,hour_168,hour_169,hour_170,hour_171,hour_172,hour_173,hour_174,hour_175,hour_176,hour_177,hour_178,hour_179,hour_180,hour_181,hour_182,hour_183,hour_184,hour_185,hour_186,hour_187,hour_188,hour_189,hour_190,hour_191,hour_192,hour_193,hour_194,hour_195,hour_196,hour_197,hour_198,hour_199,hour_200,hour_201,hour_202,hour_203,hour_204,hour_205,hour_206,hour_207,hour_208,hour_209,hour_210,hour_211,hour_212,hour_213,hour_214,hour_215,hour_216,hour_217,hour_218,hour_219,hour_220,hour_221,hour_222,hour_223,hour_224,hour_225,hour_226,hour_227,hour_228,hour_229,hour_230,hour_231,hour_232,hour_233,hour_234,hour_235,hour_236,hour_237,hour_238,hour_239,hour_240,hour_241,hour_242,hour_243,hour_244,hour_245,hour_246,hour_247,hour_248,hour_249,hour_250,hour_251,hour_252,hour_253,hour_254,hour_255,hour_256,hour_257,hour_258,hour_259,hour_260,hour_261,hour_262,hour_263,hour_264,hour_265,hour_266,hour_267,hour_268,hour_269,hour_270,hour_271,hour_272,hour_273,hour_274,hour_275,hour_276,hour_277,hour_278,hour_279,hour_280,hour_281,hour_282,hour_283,hour_284,hour_285,hour_286,hour_287,hour_288,hour_289,hour_290,hour_291,hour_292,hour_293,hour_294,hour_295,hour_296,hour_297,hour_298,hour_299,hour_300,hour_301,hour_302,hour_303,hour_304,hour_305,hour_306,hour_307,hour_308,hour_309,hour_310,hour_311,hour_312,hour_313,hour_314,hour_315,hour_316,hour_317,hour_318,hour_319,hour_320,hour_321,hour_322,hour_323,hour_324,hour_325,hour_326,hour_327,hour_328,hour_329,hour_330,hour_331,hour_332,hour_333,hour_334,hour_335,hour_336,hour_337,hour_338,hour_339,hour_340,hour_341,hour_342,hour_343,hour_344,hour_345,hour_346,hour_347,hour_348,hour_349,hour_350,hour_351,hour_352,hour_353,hour_354,hour_355,hour_356,hour_357,hour_358,hour_359,hour_360,hour_361,hour_362,hour_363,hour_364,hour_365,hour_366,hour_367,hour_368,hour_369,hour_370,hour_371,hour_372,hour_373,hour_374,hour_375,hour_376,hour_377,hour_378,hour_379,hour_380,hour_381,hour_382,hour_383,hour_384,hour_385,hour_386,hour_387,hour_388,hour_389,hour_390,hour_391,hour_392,hour_393,hour_394,hour_395,hour_396,hour_397,hour_398,hour_399,hour_400,hour_401,hour_402,hour_403,hour_404,hour_405,hour_406,hour_407,hour_408,hour_409,hour_410,hour_411,hour_412,hour_413,hour_414,hour_415,hour_416,hour_417,hour_418,hour_419,hour_420,hour_421,hour_422,hour_423,hour_424,hour_425,hour_426,hour_427,hour_428,hour_429,hour_430,hour_431,hour_432,hour_433,hour_434,hour_435,hour_436,hour_437,hour_438,hour_439,hour_440,hour_441,hour_442,hour_443,hour_444,hour_445,hour_446,hour_447,hour_448,hour_449,hour_450,hour_451,hour_452,hour_453,hour_454,hour_455,hour_456

RAW_STOP_COUNTS is large than the sum(stop) in "DEVICE_HOME_AREAS", result_df['stop'].sum() / result_df['RAW_STOP_COUNTS'].sum() = 85%

The DEVICE_HOME_AREAS column report less CBGs # thant the RAW_STOP_COUNTS column.

### verify the relationship between raw_device_count and the sum of 720 hours.

In [17]:

df = month_df.query("61 < MEDIAN_DWELL < 63")
# df = month_df.sample(300)

device_home_areas_test_df = ad_op.split_device_home_areas_stops(np_df=df.iloc[:])
result_df = device_home_areas_test_df.groupby('destination', as_index=False)[['stop', 'device']].sum().merge(df, left_on='destination', right_on='AREA').drop(columns='AREA')
# result_df = result_df[['destination', 'stop', 'RAW_STOP_COUNTS']]    # [['AREA', 'RAW_STOP_COUNTS', 'RAW_DEVICE_COUNTS']]


stop_diff = result_df['stop'].sum() / result_df['adjusted_raw_stop'].sum()
print("stop_diff ratio: ", stop_diff)

adjusted_diff = result_df['adjusted_raw_stop'].sum() / result_df['RAW_STOP_COUNTS'].sum()
print("adjusted stop diff ratio: ", adjusted_diff)

device_diff = result_df['device'].sum() / result_df['RAW_DEVICE_COUNTS'].sum()
print("DEVICE_HOME_AREAS column report less CBGs in RAW_DEVICE_COUNTS column （ratio）: ", device_diff)
result_df.head()

3911it [00:01, 3091.16it/s]


Merging columns...
Stop factor:  1
stop_diff ratio:  5.688777570312771
adjusted stop diff ratio:  2.0
DEVICE_HOME_AREAS column report less CBGs in RAW_DEVICE_COUNTS column （ratio）:  1.0


,destination,stop,device,DEVICE_HOME_AREAS,STOPS_BY_DAY,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,MEDIAN_DWELL,STOPS_BY_EACH_HOUR,WORK_BEHAVIOR_DEVICE_HOME_AREAS,assumed_stops,total_work_device,total_hourly_stop,adjusted_raw_stop
0,010010208024,375968.0,6064.0,"{""010010208024"":402,""010010209004"":200,""010010...","[1446,1259,1241,1222,1265,1366,1221,1368,1386,...",42407,6064,62.616667,"[20,4,3,2,3,20,27,53,52,58,75,82,100,110,110,1...","{""010010208024"":317,""010510313001"":9,""01051031...",375968,449,42407,84814
1,010150015001,175274.0,2827.0,"{""010150015001"":230,""010150015002"":116,""011210...","[898,605,543,545,809,900,869,933,589,587,941,1...",24980,2827,61.816667,"[14,10,5,13,5,35,45,31,46,64,55,72,48,57,62,57...","{""010150015001"":169,""010150008001"":7,""01015001...",175274,278,24980,49960
2,010150017001,200074.0,3227.0,"{""010150017001"":149,""010150017002"":138,""010150...","[569,694,480,441,507,529,514,520,794,536,506,5...",18485,3227,61.116667,"[6,2,1,2,3,2,8,20,16,26,24,32,37,35,32,57,39,4...","{""010150017001"":110,""010150017002"":4,""01015001...",200074,166,18485,36970
3,010150018002,164300.0,2650.0,"{""010150018002"":163,""010150018003"":139,""010150...","[635,535,535,626,530,540,553,564,488,524,574,5...",17461,2650,62.533333,"[6,2,2,0,1,2,9,27,30,29,28,38,45,42,36,37,52,6...","{""010150018002"":121,""010150008001"":4,""01015001...",164300,177,17461,34922
4,010179539001,111972.0,1806.0,"{""010179539001"":113,""010179542003"":51,""0101795...","[316,489,456,499,303,312,345,298,403,338,332,2...",10621,1806,62.500000,"[2,0,3,0,0,2,1,7,10,11,16,21,21,25,24,31,34,18...","{""010179539001"":76,""131131403062"":4,""010179546...",111972,116,10621,21242


In [18]:
# device_home_areas_df
# STOP

### Save the a single CSV for further use, only keep the non-dictionary columns.

In [19]:
non_dict_columns = ['AREA', 
                    "STOPS_BY_DAY", 
                    "RAW_STOP_COUNTS", 
                    "RAW_DEVICE_COUNTS", 
                    "MEDIAN_DWELL", 
                    "STOPS_BY_EACH_HOUR",
                   'adjusted_raw_stop',
                   ]
# save CSV file
if save_file:
    month_df[non_dict_columns].round(0).to_csv(no_dict_colum_fname, index=False)
    print("Combined CSV wihouth dictionary columns was saved at:", no_dict_colum_fname)

Combined CSV wihouth dictionary columns was saved at: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\NP_dict_column_2022_07.csv


## Split the DEVICE_HOME_AREAS column

Some CBGs are not reported because their visitors < 4, we add them according to the RAW_DEVICE_COUNTS

Note: need > 30 GB memory

In [20]:
month_df

,AREA,DEVICE_HOME_AREAS,STOPS_BY_DAY,RAW_STOP_COUNTS,RAW_DEVICE_COUNTS,MEDIAN_DWELL,STOPS_BY_EACH_HOUR,WORK_BEHAVIOR_DEVICE_HOME_AREAS,assumed_stops,total_work_device,total_hourly_stop,adjusted_raw_stop
3310,010010201001,"{""010010205002"":157,""010010208021"":138,""010010...","[394,295,266,422,348,361,394,426,298,303,355,4...",11888,2991,23.600000,"[2,0,0,0,2,2,9,30,15,18,19,34,27,31,33,31,25,3...","{""010010201001"":47,""515200201003"":4,""121319506...",185442,91,11888,11888
8832,010010201002,"{""010010201002"":156,""010010205002"":62,""0100102...","[310,268,232,298,258,296,250,303,304,272,249,2...",8906,1167,216.833333,"[4,1,3,0,0,1,4,5,10,11,9,17,35,13,23,23,26,31,...","{""010010201002"":118,""010510310001"":4,""01001020...",72354,138,8906,35624
16197,010010202001,"{""010010208022"":87,""010010205002"":82,""01001020...","[208,263,241,246,226,216,199,229,213,219,325,2...",8252,1507,46.100000,"[6,0,2,0,0,0,1,6,5,4,7,6,12,17,20,17,10,21,23,...","{""010010202001"":58,""010010205002"":4,""360290168...",93434,70,8252,8252
7306,010010202002,"{""010010205002"":439,""010010206001"":260,""010010...","[1478,985,946,1461,1310,1414,1411,1409,1113,10...",41567,8602,10.283333,"[9,2,4,2,1,12,17,67,73,107,94,103,187,109,93,9...","{""010010202002"":73,""010010208021"":9,""010510309...",533324,214,41567,41567
19512,010010203001,"{""010010203001"":310,""010010205002"":147,""010010...","[696,712,740,726,655,649,662,713,672,752,754,6...",22195,2898,141.300000,"[7,4,1,2,2,4,10,22,36,34,26,35,30,35,48,49,52,...","{""010010203001"":234,""010010205002"":4,""01001020...",179676,286,22195,66585
...,...,...,...,...,...,...,...,...,...,...,...,...
13915,780309611001,"{""780309611001"":83,""780309602003"":35,""78030960...","[308,384,152,150,440,506,319,357,337,217,338,4...",10355,2804,30.400000,"[2,1,0,0,2,4,1,16,21,26,16,32,21,22,19,23,20,2...","{""780309611001"":33,""120970408021"":5,""780309609...",173848,90,10355,10355
5025,780309611002,"{""780309611002"":51,""780309611001"":45,""78030960...","[183,147,80,103,181,173,160,238,162,105,148,17...",4946,1293,16.650000,"[5,0,0,0,2,3,3,8,8,12,13,13,20,16,8,8,13,14,10...","{""780309611002"":25,""780309605001"":4,""780209501...",80166,53,4946,4946
6825,780309612001,"{""780309612001"":45,""780309602003"":10,""78030961...","[60,49,35,49,59,44,43,51,42,44,53,59,57,71,57,...",1595,242,116.116667,"[3,0,1,0,1,0,0,3,2,3,1,3,3,5,1,1,4,6,5,7,5,2,3...","{""780309612001"":18,""780309604004"":4}",15004,22,1595,3190
21891,780309612002,"{""780309612001"":19,""780309609001"":17,""78030961...","[132,188,73,65,245,222,172,128,113,83,120,195,...",4687,1492,22.516667,"[1,0,1,0,0,1,0,6,10,13,5,5,16,9,14,8,9,12,5,1,...","{""780309612002"":5,""120190313002"":4,""7803096030...",92504,45,4687,4687


In [ ]:
device_home_areas_df = ad_op.split_device_home_areas_stops(np_df=month_df.iloc[6486:], stop_factor=stop_factor, total_stop_column = 'adjusted_raw_stop')


46385it [00:24, 1503.79it/s]

In [ ]:
ad_op.split_device_home_areas_stops(np_df=month_df.iloc[6486:], stop_factor=stop_factor, total_stop_column = 'adjusted_raw_stop')

In [ ]:
month_df.iloc[20810]

In [ ]:
month_df.iloc[6486]

In [ ]:
device_home_areas_df = device_home_areas_df.round(0)
# print("Sorting...")
## device_home_areas_df = device_home_areas_df.sort_values(['origin', 'destination'])
## device_home_areas_df = device_home_areas_df.sort_values(['origin', 'device'])
# device_home_areas_df['origin'] = device_home_areas_df['origin'].astype(str).str.zfill(12)
# device_home_areas_df['destination'] = device_home_areas_df['destination'].astype(str).str.zfill(12)
## device_home_areas_df['device'] = device_home_areas_df['device'].astype(int)

if save_file:
    print(f"Saving at: {device_home_areas_fname}...")
    device_home_areas_df.to_csv(device_home_areas_fname, index=False)
print("Done")
device_home_areas_df

In [ ]:
device_home_areas_df[['device', 'stop']].sum() / 10**9, month_df[['RAW_DEVICE_COUNTS', 'RAW_STOP_COUNTS', 'adjusted_raw_stop']].sum() / 10**9

### Save aggregation for origin and destination distribution

In [ ]:
# Save aggregation for origin and destination distribution
# if save_file:
CBG_device_stop_destination_distribution_df = device_home_areas_df.groupby('destination')[['device', 'stop']].sum().round(0)
CBG_device_stop_destination_distribution_df.round(0).to_csv(CBG_device_stop_destination_distribution_fname)

CBG_device_stop_origin_distribution_df = device_home_areas_df.groupby('origin')[['device', 'stop']].sum().round(0)
CBG_device_stop_origin_distribution_df.round(0).to_csv(CBG_device_stop_origin_distribution_fname)

CBG_device_stop_origin_distribution_df

In [ ]:
CBG_device_stop_destination_distribution_df#.reset_index()

In [ ]:
CBG_device_stop_destination_distribution_df.sum()  / 10**9, CBG_device_stop_origin_distribution_df.sum()  / 10**9

## Generate person_per_device (i.e., device sampling rate) file for origin CBGs

In [ ]:
ACS_df = pd.read_csv(ACS_file)
ACS_df["CBG_FIPS"] = ACS_df["fips"].astype(str).str.zfill(12)
print("Total US population (million):",  ACS_df['totalpopulation'].sum() / 10**6)

In [ ]:
print("Loading home panel file:", home_panel_fname)
panel_df = pd.read_csv(home_panel_fname)
panel_df

In [ ]:
monthly_device_cnt_df = panel_df.query(f"ISO_COUNTRY_CODE == 'US' ").groupby(['YEAR', 'MON'], as_index=True)['NUMBER_DEVICES_RESIDING'].sum() / 10**6
print("monthly_device_cnt_df   (million):" )
monthly_device_cnt_df.reset_index()

The NUMBER_DEVICES_RESIDING surged from 45 million to 900 million, and the sampling rate cannot be obatain, because it exceeds the US population.

In [ ]:

target_panel_df = panel_df.query(f"YEAR == {year} and MON == {month} and ISO_COUNTRY_CODE == 'US' ").sort_values('CENSUS_BLOCK_GROUP')  # 
# target_panel_df = panel_df.query(f"YEAR == {year} and MON == {6} and ISO_COUNTRY_CODE == 'US' ").sort_values('CENSUS_BLOCK_GROUP')  # 
print("Total US NUMBER_DEVICES_RESIDING (million):",  target_panel_df['NUMBER_DEVICES_RESIDING'].sum() / 10**6)
target_panel_df

In [ ]:
panel_ACS_df = target_panel_df.merge(ACS_df[['CBG_FIPS', 'totalpopulation']], left_on='CENSUS_BLOCK_GROUP', right_on='CBG_FIPS')
panel_ACS_df = panel_ACS_df.dropna(subset='NUMBER_DEVICES_RESIDING')
panel_ACS_df['NUMBER_DEVICES_RESIDING'] = panel_ACS_df['NUMBER_DEVICES_RESIDING'].astype(int)
panel_ACS_df = panel_ACS_df.sort_values('CBG_FIPS')
panel_ACS_df['sampling_rate'] = panel_ACS_df['NUMBER_DEVICES_RESIDING'] / panel_ACS_df['totalpopulation']

# panel_ACS_df['person_per_device'] = panel_ACS_df['totalpopulation'] / panel_ACS_df['NUMBER_DEVICES_RESIDING'] / (stop_sampling_rate)

# consider the stop sampling rate
# stop
# panel_ACS_df['sampling_rate'] = panel_ACS_df['sampling_rate'] ** stop_sampling_rate_exp
panel_ACS_df['sampling_rate'] = panel_ACS_df['sampling_rate'] * stop_sampling_rate

panel_ACS_df['person_per_device'] = 1 / panel_ACS_df['sampling_rate']


print("Data cover US population (million) :", panel_ACS_df['totalpopulation'].sum() / 10**6)
panel_ACS_df

In [ ]:
fig, ax = plt.subplots()
# month = 6
ax.set_title(f"Device sampling rate histogram ({year}-{month:02})")
ax.set_xlabel("Sampling rate")
ax.set_ylabel("CBG count")

panel_ACS_df.query("totalpopulation > 0  ")['sampling_rate'].dropna().hist(ax=ax, bins=np.array(range(0, 200,1))/100)

In [ ]:
# get people_per_device of a county

panel_ACS_df['county_FIPS'] = panel_ACS_df['CBG_FIPS'].str[:5]
sampling_rate_county_df = panel_ACS_df.groupby(['county_FIPS'], as_index=False).agg(    # , 'YEAR', "MON", "REGION", "ISO_COUNTRY_CODE"
    county_device=('NUMBER_DEVICES_RESIDING', 'sum'),
    county_device_daytime=('NUMBER_DEVICES_RESIDING', 'sum'),
    county_device_population=('totalpopulation', 'sum'),
    ) 

sampling_rate_county_df['person_per_device_county'] = sampling_rate_county_df['county_device_population'] /  sampling_rate_county_df['county_device']

print("National device and population count:\n")
print(sampling_rate_county_df[['county_device', 'county_device_population']].sum())

people_per_device_df = panel_ACS_df.merge(sampling_rate_county_df, left_on='county_FIPS', right_on='county_FIPS')
people_per_device_df.to_csv(person_per_device_fname, index=False)
people_per_device_df

In [ ]:
# sampling_rate

## Compute the inbounds for each destination CBG

merge the device_home_area and person_per_device, then groupby the destination

In [ ]:
# device_home_areas_df = pd.read_csv(device_home_areas_fname, dtype={"origin":str, "destination":str})
device_home_areas_df

In [ ]:
# people_per_device_df = pd.read_csv(person_per_device_fname, index=False)

print("Loading people_per_device_df from:", person_per_device_fname)
person_per_device_df = pd.read_csv(person_per_device_fname, dtype={"CBG_FIPS": str})

print("Total totalpopulation (billion):", person_per_device_df['totalpopulation'].sum() / 10**9)
person_per_device_df

In [ ]:
origin_person_per_device_df = device_home_areas_df.merge(person_per_device_df[['CBG_FIPS', 'person_per_device', 'person_per_device_county']], left_on="origin", right_on="CBG_FIPS")
origin_person_per_device_df

In [ ]:
origin_person_per_device_df['person_CBG_rate'] = origin_person_per_device_df['device'] * origin_person_per_device_df['person_per_device']
origin_person_per_device_df['person_county_rate'] = origin_person_per_device_df['device'] * origin_person_per_device_df['person_per_device_county']
origin_person_per_device_df[['person_CBG_rate','person_county_rate']].sum() / 10**9

In [ ]:
origin_person_per_device_df[['person_CBG_rate', 'device','stop']].sum() / 10**9

In [ ]:
def get_inbound_person_per_device(df):
    inboud_per_per_device_CBG_rate = df['person_CBG_rate'].sum() /  df['device'].sum()
    inboud_per_per_device_county_rate = df['person_county_rate'].sum() /  df['device'].sum()
    return {"inboud_per_per_device_CBG_rate": inboud_per_per_device_CBG_rate, "inboud_per_per_device_county_rate": inboud_per_per_device_county_rate}

CBG_destination_inbound_per_device = origin_person_per_device_df.iloc[:].groupby('destination').progress_apply(get_inbound_person_per_device)
CBG_destination_inbound_per_device_df = pd.DataFrame(CBG_destination_inbound_per_device.tolist(), index=CBG_destination_inbound_per_device.index)
CBG_destination_inbound_per_device_df.to_csv(CBG_destination_inbound_per_device_fname)
print(CBG_destination_inbound_per_device_df.mean())
CBG_destination_inbound_per_device_df

In [ ]:
print("Loading CBG_destination_inbound_per_device_df from:", CBG_destination_inbound_per_device_fname)
CBG_destination_inbound_per_device_df = pd.read_csv(CBG_destination_inbound_per_device_fname, dtype={"destination": str}).set_index('destination')

print("CBG_destination_inbound_per_device_df.mean(): \n", CBG_destination_inbound_per_device_df.mean() )
CBG_destination_inbound_per_device_df

In [ ]:
print("Loading adjusted_dwelling_time_stop_CBG_df from:", adjusted_dwelling_time_stop_CBG_fname)
# adjusted_dwelling_time_stop_CBG_df = pd.read_csv(adjusted_dwelling_time_stop_CBG_fname, dtype={"CBG": str}).set_index('CBG')

print("Total adjusted hourly stops (billion):", adjusted_dwelling_time_stop_CBG_df.iloc[:, 1:].sum().sum() / 10**9)
# try:
adjusted_dwelling_time_stop_CBG_df.set_index('CBG', inplace=True)

In [ ]:
adjusted_dwelling_time_stop_inbound_per_device_CBG_df = adjusted_dwelling_time_stop_CBG_df.merge(CBG_destination_inbound_per_device_df,left_index=True, right_index=True)
adjusted_dwelling_time_stop_inbound_per_device_CBG_df.index.name = 'CBG'
adjusted_dwelling_time_stop_inbound_per_device_CBG_df

In [ ]:
hour_cnt = adjusted_dwelling_time_stop_CBG_df.shape[1]
print("hour_cnt:", hour_cnt)

In [ ]:
# inbound_hourly_df.iloc[:, hour]
# inbound_hourly_df.loc[:, 'inboud_per_per_device_CBG_rate']

In [ ]:

inbound_hourly_df = adjusted_dwelling_time_stop_inbound_per_device_CBG_df.astype(np.float64).copy()

for hour in tqdm(range(hour_cnt)):
    # inbound_hourly_df.iloc[:, hour] = inbound_hourly_df.iloc[:, hour] * inbound_hourly_df.loc[:, 'inboud_per_per_device_CBG_rate']   # 70.4215236989238 billion
    inbound_hourly_df.iloc[:, hour] = inbound_hourly_df.iloc[:, hour] * inbound_hourly_df.loc[:, 'inboud_per_per_device_county_rate']   # 56.34291367286323 billion
print("Total restored inbound (billion) using per_device_county_rate:",  inbound_hourly_df.iloc[:, :hour].sum().sum() / 10**9)

inbound_hourly_df = adjusted_dwelling_time_stop_inbound_per_device_CBG_df.astype(np.float64).copy()

for hour in tqdm(range(hour_cnt)):
    inbound_hourly_df.iloc[:, hour] = inbound_hourly_df.iloc[:, hour] * inbound_hourly_df.loc[:, 'inboud_per_per_device_CBG_rate']   # 70.4215236989238 billion
    # inbound_hourly_df.iloc[:, hour] = inbound_hourly_df.iloc[:, hour] * inbound_hourly_df.loc[:, 'inboud_per_per_device_county_rate']   # 56.34291367286323 billion
print("Total restored inbound (billion) using per_device_CBG_rate:",  inbound_hourly_df.iloc[:, :hour].sum().sum() / 10**9)

print("We will use the larger resutls: per_device_CBG_rate.")

print("Saving inbound_hourly_df at:", inbound_hourly_df_fname)

if save_file:
    inbound_hourly_df.round(0).to_csv(inbound_hourly_df_fname)
inbound_hourly_df

In [ ]:
# Get the hourly aggregated inbounds for all CBGs
inbound_hourly_df.iloc[:, :hour_cnt].sum().sum() / 10**9

In [ ]:
hourly_agregated_inbound_df = pd.DataFrame(inbound_hourly_df.iloc[:, :hour_cnt].sum(), columns=['inboud']).reset_index()[['inboud']]

# if save_file:
hourly_agregated_inbound_df.round(0).to_csv(hourly_agregated_inbound_fname, index=False)
print("Save hourly_agregated_inbound_df at:", hourly_agregated_inbound_fname)

CBG_agregated_inbound_df = pd.DataFrame(inbound_hourly_df.iloc[:, :hour_cnt].sum(axis=1), columns=['inboud'])
CBG_agregated_inbound_df.round(0).to_csv(CBG_agregated_inbound_fname)
print("Save CBG_agregated_inbound_df at:", CBG_agregated_inbound_fname)

In [ ]:
hourly_agregated_inbound_df
hourly_agregated_inbound_df.sum() / 10**9

In [ ]:
CBG_agregated_inbound_df

In [ ]:
people_per_device_df

In [ ]:
# Load flow data (month)
# device_home_areas_df = pd.read_csv(device_home_areas_fname, dtype={'origin':str, 'destination':str})
device_home_areas_df

In [ ]:
device_home_areas_person_per_device_df = device_home_areas_df.merge(people_per_device_df[['CBG_FIPS', 'person_per_device', 'person_per_device_county']], left_on='origin', right_on='CBG_FIPS')
device_home_areas_person_per_device_df

In [ ]:
device_home_areas_person_per_device_df['stop'].sum() / 10**9, device_home_areas_person_per_device_df['device'].sum() / 10**9

In [ ]:
# Not correct
# device_home_areas_person_per_device_df['outbound'] = device_home_areas_person_per_device_df['stop'] * device_home_areas_person_per_device_df['person_per_device']
# device_home_areas_person_per_device_df['outbound'].sum()  / 10**9

In [ ]:
# no_dict_colum_month_df = pd.read_csv(no_dict_colum_fname, dtype={"AREA":str})
# no_dict_colum_month_df['destination'] = no_dict_colum_month_df['AREA'].astype(str).str.zfill(12)
# no_dict_colum_month_df

In [ ]:
# no_dict_colum_month_df['adjusted_raw_stop'].sum() / 10**9

In [ ]:
# destination_CBG_inbound_df = device_home_areas_person_per_device_df.groupby('destination', as_index=False)[['inbound']].sum()#.sum()
# destination_CBG_inbound_df['destination'] = destination_CBG_inbound_df['destination'].astype(str).str.zfill(12)

# destination_CBG_inbound_device_device_count_df = destination_CBG_inbound_df.merge(no_dict_colum_month_df[['destination', 'RAW_STOP_COUNTS', 'RAW_DEVICE_COUNTS', 'MEDIAN_DWELL']],
#                                                                                  left_on="destination",
#                                                                                  right_on="destination")

# destination_CBG_inbound_device_device_count_df['inbound_per_stop'] = destination_CBG_inbound_device_device_count_df['inbound'] / destination_CBG_inbound_device_device_count_df['RAW_STOP_COUNTS']
# destination_CBG_inbound_device_device_count_df

In [ ]:
# destination_CBG_inbound_device_device_count_df['inbound'].sum() / 1000000000, destination_CBG_inbound_device_device_count_df['RAW_STOP_COUNTS'].sum() / 1000000000

In [ ]:
# destination_CBG_inbound_device_device_count_df.to_csv(CBG_destination_inbound_fname, index=False)
# print("Saved destination_CBG_inbound_device_device_count_df as:", CBG_destination_inbound_fname)
# destination_CBG_inbound_device_device_count_df

In [ ]:
# destination_CBG_inbound_device_device_count_df['inbound'].sum() / 1000000000

## Compute the hourly inbound (not used, not correct)

In [ ]:
# destination_CBG_inbound_df = pd.read_csv(CBG_destination_inbound_fname, dtype={'destination':str})
# destination_CBG_inbound_df

In [ ]:
# destination_CBG_inbound_df.sample(30)['inbound_per_stop']#.mean()

In [ ]:
# print("Loading in adjusted_dwelling_time_stop_CBG_df from:", adjusted_dwelling_time_stop_CBG_fname)
# adjusted_dwelling_time_stop_CBG_df = pd.read_csv(adjusted_dwelling_time_stop_CBG_fname, dtype={'CBG':str})
# print("Total adjusted observed stop (billion):", adjusted_dwelling_time_stop_CBG_df.iloc[:, 1:].sum().sum() / 1000000000)
# adjusted_dwelling_time_stop_CBG_df

In [ ]:
# hour_cnt = adjusted_dwelling_time_stop_CBG_df.shape[1] - 1
# print("hour_cnt:", hour_cnt)

In [ ]:
# adjusted_stop_inbound_per_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df.merge(destination_CBG_inbound_df[['destination','inbound_per_stop']],
#                                                                                 left_on="CBG",
#                                                                                 right_on="destination",
#                                                                                 )
# adjusted_stop_inbound_per_stop_CBG_df

In [ ]:
# hourly_inbound_df = adjusted_stop_inbound_per_stop_CBG_df.copy()
# for h in range(hour_cnt):
#     hourly_inbound_df.iloc[:, h + 1] = hourly_inbound_df.iloc[:, h + 1] * hourly_inbound_df['inbound_per_stop']  

# hourly_inbound_df = hourly_inbound_df.iloc[:, :-2]
# print("Total inbound (billion):", hourly_inbound_df.iloc[:, 1:].sum().sum() / 1000000000)
# hourly_inbound_df

In [ ]:
# hourly_inbound_df.iloc[:, 1:].sum().sum()

# IPF  v2

### Restored the outbound accor
ding to device sampling rate

In [ ]:
# CBG_device_stop_origin_distribution_df = pd.read_csv(CBG_device_stop_origin_distribution_fname)
print("Total observed stops (billion): ", CBG_device_stop_origin_distribution_df['stop'].sum() / 1000000000)
CBG_device_stop_origin_distribution_df

In [ ]:
# people_per_device_df = pd.read_csv(person_per_device_fname, dtype={'CBG_FIPS': str})
people_per_device_df

In [ ]:
CBG_origin_stop_distribution_sampling_rate_df = CBG_device_stop_origin_distribution_df.reset_index().merge(people_per_device_df, left_on='origin', right_on='CBG_FIPS')
CBG_origin_stop_distribution_sampling_rate_df

Because "stop" is the combination of visitors from various HOME CBGs! So, we need to compute the inbound. 

The outbound compuated below can be view as the distribution only.

In [ ]:
person_per_device_national = CBG_origin_stop_distribution_sampling_rate_df['totalpopulation'].sum() / CBG_origin_stop_distribution_sampling_rate_df['NUMBER_DEVICES_RESIDING'].sum()
CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'] = CBG_origin_stop_distribution_sampling_rate_df['stop'] * person_per_device_national
restored_total_restored_outbound_count = CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'].sum() /  10**9
print("Restored total outbound distribution (billion) using national sampling rate:", restored_total_restored_outbound_count)  # 37.44716723845148 billion

CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'] = CBG_origin_stop_distribution_sampling_rate_df['stop'] * CBG_origin_stop_distribution_sampling_rate_df['person_per_device_county']
restored_total_restored_outbound_count = CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'].sum() /  10**9
print("Restored total outbound distribution (billion) using county sampling rate:", restored_total_restored_outbound_count)  # 50.96843487672891 billion

CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'] = CBG_origin_stop_distribution_sampling_rate_df['stop'] * CBG_origin_stop_distribution_sampling_rate_df['person_per_device']
restored_total_restored_outbound_count = CBG_origin_stop_distribution_sampling_rate_df['restored_outbound_count'].sum() / 10**9
CBG_origin_stop_distribution_sampling_rate_df.to_csv(CBG_origin_outbound_distribution_sampling_rate_fname, index=False)
print("Restored total outbound distribution (billion) using CBG sampling rate:", restored_total_restored_outbound_count)  # 61.19985609714293 billion

print("Saved CBG_origin_outbound_distribution_sampling_rate at:", CBG_origin_outbound_distribution_sampling_rate_fname)

In [ ]:
# use the total_inboud to calibrate the outbound

# restored_outbound_countCBG_origin_stop_distribution_sampling_rate_df
# CBG_origin_stop_distribution_sampling_rate_df
CBG_origin_stop_distribution_sampling_rate_df

## Load data

In [ ]:
hourly_agregated_inbound_df = pd.read_csv(hourly_agregated_inbound_fname)
total_inbound = hourly_agregated_inbound_df['inboud'].sum()
print("total_inbound (billion):", total_inbound / 10**9)

In [ ]:
# hourly_agregated_inbound_df

In [ ]:
# CBG_restored_origin_outbound_df = pd.read_csv(CBG_origin_outbound_distribution_sampling_rate_fname)
CBG_restored_origin_outbound_df = CBG_origin_stop_distribution_sampling_rate_df
total_outbound = CBG_restored_origin_outbound_df['restored_outbound_count'].sum()
print("Total outbound distribution (billions): ",  total_outbound / 1000000000)


factor = max(total_inbound, total_outbound) / min(total_inbound, total_outbound)
print("factor:", factor)


CBG_restored_origin_outbound_df['restored_outbound_count'] = CBG_restored_origin_outbound_df['restored_outbound_count'] * factor

print("Total outbound distribution (billions): ",   CBG_restored_origin_outbound_df['restored_outbound_count'].sum() / 10**9)

In [ ]:
CBG_restored_origin_outbound_df.shape

In [ ]:
# CBG_restored_origin_outbound_df

In [ ]:
# Load 

In [ ]:
# adjusted_dwelling_time_stop_natinol_df = pd.read_csv(adjusted_dwelling_time_stop_natinol_fname)
# total_adjusted_stops = adjusted_dwelling_time_stop_natinol_df['stop'].sum()
# print("total_adjusted_stops (billion):", total_adjusted_stops / 1000000000)

# # covnert distribution to acution outbound
# outbound_per_stop = total_outbound / total_adjusted_stops 

# adjusted_dwelling_time_stop_natinol_df['stop'] *= outbound_per_stop
# print("total outbouds (billion):", adjusted_dwelling_time_stop_natinol_df['stop'].sum() / 1000000000)

### start IPF

In [ ]:
# restored_CBG_outbound_arr.sum() / 10**9
# restored_hourly_outbound_arr.sum() / 10**9

In [ ]:
import numpy as np
from ipfn import ipfn

restored_CBG_outbound_arr = CBG_restored_origin_outbound_df['restored_outbound_count'].to_numpy()
restored_hourly_outbound_arr = hourly_agregated_inbound_df['inboud'].to_numpy()  # 

aggregates = [restored_CBG_outbound_arr, restored_hourly_outbound_arr]
dimensions = [[0], [1]]

seed_arr = np.ones((len(restored_CBG_outbound_arr), len(restored_hourly_outbound_arr)))

IPF = ipfn.ipfn(seed_arr, aggregates, dimensions, convergence_rate=1e-3)
m = IPF.iteration()
print(m.shape)

In [ ]:
print("Error of IPF:")
print((m.sum(axis=1) - restored_CBG_outbound_arr).sum(), (m.sum(axis=0) - restored_hourly_outbound_arr).sum())

In [ ]:
(m.sum(axis=1) - restored_CBG_outbound_arr).max(), (m.sum(axis=0) - restored_hourly_outbound_arr).max()

In [ ]:
(m.sum(axis=1) - restored_CBG_outbound_arr).min(), (m.sum(axis=0) - restored_hourly_outbound_arr).min()

In [ ]:
# Save IPF results
hourly_columns = [f'hour_{h+1}' for h in range(restored_hourly_outbound_arr.shape[0])]
IPF_df = pd.DataFrame(m, columns=hourly_columns)
print("Total outbound visitor (billion):", IPF_df.sum().sum() / 1000000000)

 
IPF_df['CBG'] = CBG_restored_origin_outbound_df['origin'].astype(str).str.zfill(12)
IPF_df = IPF_df[['CBG'] + hourly_columns]#.set_index('CBG')

if save_file:
    print("Saving IPF at:", IPF_results_fname)
    IPF_df.round(0).to_csv(IPF_results_fname, index=True)
    print("Saved IPF_results at:", IPF_results_fname)
IPF_df

In [ ]:
# Not use below

##	Adjust stop count by dwelling time

Load the raw stop data and Neighorhood Patterns table.

In [ ]:
# Load the raw stop data and Neighorhood Patterns table.

# device_home_areas_df = pd.read_csv(device_home_areas_fname)
# print("Loading no_dict_colum CSV file:", no_dict_colum_fname)
# no_dict_colum_df = pd.read_csv(no_dict_colum_fname)

# no_dict_colum_df

In [ ]:
# non_adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=no_dict_colum_df.iloc[:], adjust_dwell_time=False, clean_negative=True)
# print("sum of non_adjusted_dwell_time_stop_CBG_arr (million):", non_adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000)
# print("shape of non_adjusted_dwell_time_stop_CBG_arr:", non_adjusted_dwell_time_stop_CBG_arr.shape)

In [ ]:
# adjusted_dwell_time_stop_CBG_arr = ad_op.adjust_stop_by_dwelling_time(np_df=no_dict_colum_df.iloc[:], adjust_dwell_time=True, clean_negative=True)

# adjusted_dwell_time_stop_national_arr = adjusted_dwell_time_stop_CBG_arr.sum(axis=0)

# print("sum of adjusted_dwell_time_stop_CBG_arr (million):", adjusted_dwell_time_stop_CBG_arr.sum().sum()/1000000)
# print("sum of adjusted_dwell_time_stop_national_arr (million):", adjusted_dwell_time_stop_national_arr.sum().sum()/1000000)
# print("shape of adjusted_dwell_time_stop_CBG_arr:", adjusted_dwell_time_stop_CBG_arr.shape)

In [ ]:
# hourly_columns
# adjusted_dwell_time_stop_national_arr.shape

Convert numpy array to DataFrame

In [ ]:
# convert numpy array to DataFrame
# # CBG level
# hourly_columns = [f'hour_{h+1}' for h in range(adjusted_dwell_time_stop_CBG_arr.shape[1])]
# adjusted_dwelling_time_stop_CBG_df = pd.DataFrame(adjusted_dwell_time_stop_CBG_arr, columns=hourly_columns)
# adjusted_dwelling_time_stop_CBG_df['CBG'] = no_dict_colum_df['AREA'].astype(str).str.zfill(12).to_list()
# adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df[['CBG'] + hourly_columns]  # reorder columns
# adjusted_dwelling_time_stop_CBG_df = adjusted_dwelling_time_stop_CBG_df.sort_values('CBG')


# # national level
# adjusted_dwelling_time_stop_natinol_df = pd.DataFrame(adjusted_dwell_time_stop_national_arr, columns=['stop'])

# adjusted_dwelling_time_stop_CBG_df

In [ ]:
# print("Adjustted total stop (billion) by dwelling time from CBG and national level (should be the same):")
# print(adjusted_dwelling_time_stop_CBG_df[hourly_columns].sum().sum() / 1000000000,  adjusted_dwelling_time_stop_natinol_df.sum().sum()/ 1000000000)

In [ ]:
# # Save CSV files
# adjusted_dwelling_time_stop_natinol_df.to_csv(adjusted_dwelling_time_stop_natinol_fname, index=False)
# print("Saved national hourly stop at:", adjusted_dwelling_time_stop_natinol_fname)

# adjusted_dwelling_time_stop_CBG_df.to_csv(adjusted_dwelling_time_stop_CBG_fname, index=False)
# print("Saved CBG hourly stop at:", adjusted_dwelling_time_stop_CBG_fname)

###  Convert observed stop counts to actual visitor counts.

#### compute the person_per_device at CBG and county level

Load ACS and home panel data, merge them into device sampling rate table at CBG and county level

In [ ]:
# ACS_df = pd.read_csv(ACS_file)
# ACS_df["CBG_FIPS"] = ACS_df["fips"].astype(str).str.zfill(12)
# print("Total US population (million):",  ACS_df['totalpopulation'].sum() / 1000000)

In [ ]:
# print("Loading home panel file:", home_panel_fname)
# panel_df = pd.read_csv(home_panel_fname)
# target_panel_df = panel_df.query(f"YEAR == {year} and MON == {int(month)} and ISO_COUNTRY_CODE == 'US' ")
# target_panel_df

In [ ]:
# panel_ACS_df = target_panel_df.merge(ACS_df[['CBG_FIPS', 'totalpopulation']], left_on='CENSUS_BLOCK_GROUP', right_on='CBG_FIPS')
# panel_ACS_df = panel_ACS_df.dropna(subset='NUMBER_DEVICES_RESIDING')
# panel_ACS_df['NUMBER_DEVICES_RESIDING'] = panel_ACS_df['NUMBER_DEVICES_RESIDING'].astype(int)
# panel_ACS_df = panel_ACS_df.sort_values('CBG_FIPS')
# panel_ACS_df['person_per_device'] = panel_ACS_df['totalpopulation'] / panel_ACS_df['NUMBER_DEVICES_RESIDING']
# print("Data cover US population:", panel_ACS_df['totalpopulation'].sum())
# panel_ACS_df

In [ ]:
# get people_per_device of a county

# panel_ACS_df['county_FIPS'] = panel_ACS_df['CBG_FIPS'].str[:5]
# sampling_rate_county_df = panel_ACS_df.groupby(['county_FIPS'], as_index=False).agg(    # , 'YEAR', "MON", "REGION", "ISO_COUNTRY_CODE"
#     county_device=('NUMBER_DEVICES_RESIDING', 'sum'),
#     county_device_daytime=('NUMBER_DEVICES_RESIDING', 'sum'),
#     county_device_population=('totalpopulation', 'sum'),
#     ) 

# sampling_rate_county_df['person_per_device_county'] = sampling_rate_county_df['county_device_population'] /  sampling_rate_county_df['county_device']

# print("National device and population count:\n")
# print(sampling_rate_county_df[['county_device', 'county_device_population']].sum())

# people_per_device_df = panel_ACS_df.merge(sampling_rate_county_df, left_on='county_FIPS', right_on='county_FIPS')
# people_per_device_df.to_csv(person_per_device_fname, index=False)
# people_per_device_df

#### restore the observated stop count to actual visitor count

In [ ]:
# people_per_device_df = pd.read_csv(person_per_device_fname, dtype={'CBG_FIPS': str})
# adjusted_dwelling_time_stop_CBG_df = pd.read_csv(adjusted_dwelling_time_stop_CBG_fname, dtype={'CBG':str})

In [ ]:
# adjusted_dwelling_time_stop_CBG_df
# people_per_device_df

In [ ]:
# adjusted_dwelling_time_stop_CBG_df.shape

## Get the total outgoer

In [ ]:
# device_home_areas_df = pd.read_csv(device_home_areas_fname)
# device_home_areas_df

In [ ]:
# # Assume a stop is a device
# hour_cnt = adjusted_dwelling_time_stop_CBG_df.shape[1] - 1


# hourly_stop_sampling_rate_df = adjusted_dwelling_time_stop_CBG_df.merge(people_per_device_df, left_on='CBG', right_on='CBG_FIPS')
# national_people_per_device = hourly_stop_sampling_rate_df['totalpopulation'].sum() / hourly_stop_sampling_rate_df['NUMBER_DEVICES_RESIDING'].sum()

# print("hour_cnt:", hour_cnt)
# print("national_people_per_device:", national_people_per_device)

# for h in range(hour_cnt):
#     hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * hourly_stop_sampling_rate_df['person_per_device']   # CBG leval   # BUG: person_per_device is for home(origin), not for destination!!!!!!!!!!!!
#     # hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * national_people_per_device    # national leval
#     # hourly_stop_sampling_rate_df.iloc[:, h + 1] = hourly_stop_sampling_rate_df.iloc[:, h + 1] * hourly_stop_sampling_rate_df['person_per_device_county']   # county level
    

# ## used      CBG level sampling rate: 51028323628.24674
# ## used   county level sampling rate: 51223351774.32022   # increase 0.3%
# ## used national level sampling rate: 37518194037.32022   # decrease 26.5%

# restored_actual_CBG_hourly_visitor_df = hourly_stop_sampling_rate_df.iloc[:, :hour_cnt + 1]

# print("Restored total visitor count (billion):")
# print(restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum().sum() / 1000000000)  # used CBG level sampling rate: 51028323628.24674

# print("Restored total visitor dataframe shape:",  restored_actual_CBG_hourly_visitor_df.shape)

In [ ]:
# restored_actual_CBG_hourly_visitor_df

In [ ]:
# print("Restored actuall CBG hourly visitor (billion):")
# print(restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum().sum()/ 1000000000)

In [ ]:
# restored_actual_target_hourly_visitor_df = pd.DataFrame()
# restored_actual_target_hourly_visitor_df['stop'] = restored_actual_CBG_hourly_visitor_df.iloc[:, 1:hour_cnt + 1].sum(axis=0) 
# print("Restored actuall target hourly visitor (billion):")
# print(restored_actual_target_hourly_visitor_df['stop'].sum() / 1000000000)
# restored_actual_target_hourly_visitor_df

In [ ]:
# save the restored actual visitor count
# restored_actual_CBG_hourly_visitor_df.to_csv(restored_CBG_hourly_vistor_count_fname, index=False)
# restored_actual_target_hourly_visitor_df.to_csv(restored_target_hourly_vistor_count_fname, index=False)

## IPF

### Load data

In [ ]:
# restored_actual_target_hourly_visitor_df = pd.read_csv(restored_target_hourly_vistor_count_fname, dtype={'CBG': str})
# print("Restored target total visitor count (billion):")
# print(restored_actual_target_hourly_visitor_df['stop'].sum() / 1000000000) 

# restored_actual_target_hourly_visitor_df

Load the CBG stop distribution and compute the CBG aggregated restored visitors.

How to convert the CBG device distribution to stop distribution?

- option 1: convert the CBG device distribution to people distribution using device sampling rate, then using the people distribution as the stop distribution.
- option 2: compute the stop_per_device for each CBG, and then the stop for each home CBG, then aggregate the home CBG stops.

We use option 2 here.

In [ ]:
# CBG_device_stop_origin_distribution_df = pd.read_csv(CBG_device_stop_origin_distribution_fname, dtype={'origin':str})
# CBG_device_stop_origin_distribution_df

# print("origin observed stop count (billion):")
# print(CBG_device_stop_origin_distribution_df['stop'].sum() / 1000000000)

In [ ]:
# df = CBG_device_stop_origin_distribution_df.merge(people_per_device_df, left_on='origin', right_on='CBG_FIPS')
# df['restored_outgoer'] = df['stop'] * df['person_per_device']
# df['restored_outgoer'].sum() / 1000000000

In [ ]:



# visitor_per_stop = restored_actual_target_hourly_visitor_df['stop'].sum() / CBG_device_stop_origin_distribution_df['stop'].sum() 
# print("visitor_per_stop:", visitor_per_stop)

# restored_CBG_origin_outgoer_df = CBG_device_stop_origin_distribution_df[['origin', 'stop']].copy()
# restored_CBG_origin_outgoer_df['stop'] = CBG_device_stop_origin_distribution_df['stop'] * visitor_per_stop

# print("restored CBG origin outgoer count (billion):")
# restored_CBG_origin_outgoer_df['stop'].sum() / 1000000000

# restored_CBG_origin_outgoer_df

In [ ]:
# import numpy as np
# from ipfn import ipfn

# restored_outgoer_arr = restored_CBG_origin_outgoer_df['stop'].to_numpy()
# restored_actual_target_hourly_visitor_arr = restored_actual_target_hourly_visitor_df['stop'].to_numpy()

# aggregates = [restored_outgoer_arr, restored_actual_target_hourly_visitor_arr]
# dimensions = [[0], [1]]

# seed_arr = np.ones((len(restored_outgoer_arr), len(restored_actual_target_hourly_visitor_arr)))

# IPF = ipfn.ipfn(seed_arr, aggregates, dimensions, convergence_rate=1e-3)
# m = IPF.iteration()
# print(m.shape)

In [ ]:
# print(m.shape, restored_outgoer_arr.sum(), restored_actual_target_hourly_visitor_arr.sum())

# (m.sum(axis=1) - restored_outgoer_arr).sum(), (m.sum(axis=0) - restored_actual_target_hourly_visitor_arr).sum()

In [ ]:
# Save IPF results
# hourly_columns = [f'hour_{h+1}' for h in range(restored_actual_target_hourly_visitor_arr.shape[0])]
# IPF_df = pd.DataFrame(m, columns=hourly_columns)
# print("Total visitor (billion):", IPF_df.sum().sum() / 1000000000)

 
# IPF_df['CBG'] = restored_CBG_origin_outgoer_df['origin']
# IPF_df = IPF_df[['CBG'] + hourly_columns]

# IPF_df.to_csv(IPF_results_fname, index=False)
# IPF_df

## Compute hourly population map

# Load the CBG population

In [ ]:
ACS_df = pd.read_csv(person_per_device_fname, dtype={"CBG_FIPS":str, "county_FIPS":str})
ACS_df

In [ ]:
# IPF_df = pd.read_csv(IPF_results_fname, dtype={'CBG':str}).set_index('CBG')
IPF_df.set_index("CBG", inplace=True)
IPF_df

In [ ]:
hour_cnt = IPF_df.shape[1] # - 1 # when CBG is not the index 
print(f"hour cnt: {hour_cnt}")

print("IPF total outbound (billion):", IPF_df.sum().sum()/ 10**9)


In [ ]:
inbound_hourly_df_fname

In [ ]:
# try:
#     restored_actual_CBG_hourly_visitor_df = pd.read_csv(inbound_hourly_df_fname, encoding='utf-8', dtype={'CBG':str}).set_index('CBG')
# except:
#     restored_actual_CBG_hourly_visitor_df = pd.read_csv(inbound_hourly_df_fname, encoding='utf-16', dtype={'CBG':str}).set_index('CBG')

restored_actual_CBG_hourly_visitor_df = inbound_hourly_df

print("Total restored visitor count (billion):", restored_actual_CBG_hourly_visitor_df.sum().sum() / 10**9)
restored_actual_CBG_hourly_visitor_df

Find the common CBGs between IPF and ACS CBGs

In [ ]:
CBG_device_stop_origin_distribution_df = pd.read_csv(CBG_device_stop_origin_distribution_fname, dtype={'origin':str})
CBG_device_stop_origin_distribution_df

In [ ]:
CBGs = set(IPF_df.index).intersection(set(ACS_df['CBG_FIPS']))
CBGs = CBGs.intersection(set(CBG_device_stop_origin_distribution_df['origin'])).intersection(set(restored_actual_CBG_hourly_visitor_df.index))
CBGs = list(CBGs)
print("Found CBG count:", len(CBGs))

In [ ]:
filtered_ACS_df = ACS_df.set_index("CBG_FIPS").loc[CBGs].sort_index()

filtered_outgoer_df = IPF_df.loc[CBGs].sort_index()
filtered_visitor_df = restored_actual_CBG_hourly_visitor_df.loc[CBGs].iloc[:, :hour_cnt]

print("Filtered restored outgoer count (billion):", filtered_outgoer_df.sum().sum() / 1000000000)

print("Filtered restored visitor count (billion):", filtered_visitor_df.sum().sum() / 1000000000)

filtered_outgoer_df
filtered_visitor_df

In [ ]:
hourly_population_base_df = filtered_outgoer_df.copy()
duplicated_arrays = np.tile(filtered_ACS_df['totalpopulation'].to_numpy(), (hour_cnt, 1)).T

hourly_population_base_df.iloc[:, :] = duplicated_arrays
hourly_population_base_df

In [ ]:
hourly_population_map_df = hourly_population_base_df - filtered_outgoer_df + filtered_visitor_df 

print("Saving hourly map at: ", CBG_hourly_population_fname)
hourly_population_map_df.round(0).to_csv(CBG_hourly_population_fname)


hourly_population_map_df

In [ ]:
filtered_visitor_df#.sum(axis=0)

In [ ]:
filtered_outgoer_df#.sum(axis=0)

# hourly_population_df = filtered_IPF_df.copy()
# for c in hourly_population_df.columns:
#     hourly_population_df[c] = filtered_population_df['totalpopulation'].to_list()

In [ ]:
hourly_population_map_df.sum(axis=0)

In [ ]:
# n = 10
# hourly_population_map_df = hourly_population_base_df.iloc[:n] - filtered_outgoer_df.iloc[:n] + filtered_visitor_df.iloc[:n] 

## Show hourly population map

In [ ]:
# Load data
# hourly_population_map_df = pd.read_csv(CBG_hourly_population_fname, dtype={'CBG':str})
hourly_population_map_df

In [ ]:
ACS_df = pd.read_csv(person_per_device_fname, dtype={"CBG_FIPS":str, "county_FIPS":str})
ACS_df['county_FIPS'] = ACS_df['county_FIPS'].astype(str).str.zfill(5)
ACS_df

In [ ]:
def get_county_FIPS_dict():
    county_gdf = gpd.read_file(r'https://github.com/gladcolor/spatial_data/raw/master/cb_2019_us_county_20m.zip')
    state_gdf = gpd.read_file(r'https://github.com/gladcolor/spatial_data/raw/master/cb_2019_us_state_20m.zip')
    county_gdf = county_gdf.merge(state_gdf[['STATEFP', 'STUSPS', 'NAME']], left_on='STATEFP', right_on='STATEFP', suffixes=['_county', '_state'])
    county_gdf['county_state'] = county_gdf['NAME_county'] + ', ' + county_gdf['STUSPS']
    county_gdf['GEOID'] = county_gdf['GEOID'].astype(str).str.zfill(5)
    county_FIPS_dict = county_gdf.set_index(['county_state'])['GEOID'].to_dict()
    return county_FIPS_dict

county_FIPS_dict = get_county_FIPS_dict()
# county_FIPS_dict

In [ ]:
CBG2019_gdf = gpd.read_file(CBG_2019_fname)
CBG2019_gdf['county_FIPS'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12).str[:5]
CBG2019_gdf['CBG'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12) 


In [ ]:
CBG2019_gdf

In [ ]:
hourly_population_map_df.iloc[:, :hour_cnt ]

In [ ]:
import warnings

# Suppress all future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

cmap = 'viridis'
# day = 18

target_county = 'New York, NY'
target_county = 'Centre, PA'
target_county = 'Richland, SC'
target_county = 'Lexington, SC'
target_county = 'Union, NJ'
target_county = 'Fairfax, VA'
target_county = 'Los Angeles, CA'
target_county = 'Cook, IL'
target_county = 'Harris, TX'
target_county = 'Maricopa, AZ'
target_county = 'San Diego, CA'
target_county = 'Orange, CA'
target_county = 'Miami-Dade, FL'
target_county = 'Dallas, TX'
target_county = 'King, WA'
target_county = 'Riverside, CA'
target_county = 'Hudson, NJ'
target_county = 'Essex, NJ'
target_county = 'Berkshire, MA'


def draw_county_map(target_county):
    target_county = 'New York, NY'
    county_FIPS = county_FIPS_dict[target_county]
    
    target_counties = [county_FIPS] # ['36061']   # 06039
    hourly_population_map_df['county_FIPS'] = hourly_population_map_df.reset_index()['CBG'].astype(str).str[:5].to_list()
    target_hourly_pop_df = hourly_population_map_df[hourly_population_map_df['county_FIPS'].isin(target_counties)]
    
    target_pop_df = ACS_df[ACS_df['county_FIPS'].isin(target_counties)]
    
    
    target_blockgroup_gdf = CBG2019_gdf[CBG2019_gdf['county_FIPS'].isin(target_counties)]
    target_blockgroup_gdf
    
    row_cnt = 2
    col_cnt = 13
    # vmax = max(target_hourly_pop_df.iloc[:, :hour_cnt].max(axis=1).max(), target_pop_df['totalpopulation'].max())
    # vmax = target_hourly_pop_df.iloc[:, :hour_cnt ].mean(axis=1).mean() + target_hourly_pop_df.iloc[:, :hour_cnt].mean(axis=1).std() * 3
    # print("target_hourly_pop_df:\n", target_hourly_pop_df.iloc[:, 1: hour_cnt + 1].mean(axis=1).mean())
    # print(target_hourly_pop_df)
    
    vmax = target_hourly_pop_df.iloc[:, : hour_cnt].mean(axis=1).mean() + target_hourly_pop_df.iloc[:, :hour_cnt ].mean(axis=1).std() * 3
    
    print("vmax: ", vmax)
    fig, axs = plt.subplots(figsize=(40, 10), nrows=row_cnt, ncols=col_cnt)
    
    # draw hourly map
    used_ax_cnt = 0
    hour_24_pop = []
    for row in range(row_cnt):
        for col in range(1, col_cnt, 1):
            # if used_ax_cnt == 24:
            #     continue
            merged_gdf = target_blockgroup_gdf.merge(target_hourly_pop_df, left_on='CBG', right_on='CBG')
            hour = used_ax_cnt
            ax=axs[row, col]
            ax.axis('off')
            ax.set_title(f"{hour:02}:00:00")
            hour_column = f'hour_{hour + 1 + 24*(day - 1)}'
            merged_gdf.plot(column=hour_column, ax=ax, vmax=vmax, cmap=cmap)  # , vmax=vmax ,  legend=True
            # merged_gdf.plot(column=hour_column, ax=ax, cmap=cmap)
            hour_24_pop.append(target_hourly_pop_df[hour_column].sum())
            used_ax_cnt += 1
    
    ax=axs[0, 0]
    ax.axis('off')
    ax.set_title(f"ACS 2019 Population")
    merged_gdf = target_blockgroup_gdf.merge(target_pop_df, left_on='CBG', right_on='CBG_FIPS')
    # merged_gdf.plot(column='totalpopulation', ax=ax,  cmap=cmap) # , vmax=vmax ,  legend=True
    merged_gdf.plot(column='totalpopulation', ax=ax, vmax=vmax, cmap=cmap)
    
    # draw county-level population
    ax=axs[1, 0]
    # ax.axis('off')
    ax.set_title('County hourly population')
    ax.set_ylabel('Population')
    ax.set_xlabel('Hour')
    ax.plot(hour_24_pop, label='hourly population')
    ax.axhline(merged_gdf['totalpopulation'].sum(), color='red', label='ACS population')
    ax.legend()
    
     
    fig.suptitle(f"Hourly population of county {target_county}: {year}-{month:02}-{day:02}", fontsize=24, y=1)
    
    
    # Add colorbar axes at the bottom and align it with the left and right of the subplots
    pos1 = axs[0, 0].get_position() # get the original position for first axis
    pos2 = axs[-1, -1].get_position()
    cax = fig.add_axes([pos1.x0 * 1.01, 0, (pos2.x1 - pos1.x0) * 0.99, 0.008])    # [left, bottom, width, height]  # bottom
    value_max =vmax
    value_min = 0
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=value_min, vmax=value_max))
    sm._A = []
    # plt.colorbar(sm, cax=cax, label="Bias")
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
    
    for t in cbar.ax.get_xticklabels():
        t.set_fontsize(16)
    
    cbar.set_label('Population', labelpad=-80,   rotation=0, fontsize=24, loc='center')
    
    png_name = os.path.join(save_dir, f"{target_county.replace(', ', '_')}_hourly_population_{year}{month:02}{day:02}_v3.png")
    
    print("PNG name:", png_name)
    
    plt.savefig(png_name, dpi=300, bbox_inches='tight', pad_inches=0.2)
    plt.close()

for target_county, FIPS in tqdm(county_FIPS_dict.items()):
    print(target_county)
    for day in range(1, 32):
        draw_county_map(target_county)
    break

print("Done")

In [ ]:
target_hourly_pop_df
merged_gdf
hourly_population_map_df
target_blockgroup_gdf
target_county

In [ ]:
STOP

# Extract data for South Carolina

In [ ]:
state_FIPS = '45'
year = '2023'
month = 6

houry_population_fname = os.path.join(save_dir, 'for_South_Carolina', f'state_FIPS_{state_FIPS:02}_hourly_population_{year}{month:02}.csv')
print('houry_population_fname:', houry_population_fname)

inbound_fname = os.path.join(save_dir, 'for_South_Carolina', f'state_FIPS_{state_FIPS:02}_hourly_inbound_{year}{month:02}.csv')
print('inbound_fname:', inbound_fname)

outbound_fname = os.path.join(save_dir, 'for_South_Carolina', f'state_FIPS_{state_FIPS:02}_hourly_outbound_{year}{month:02}.csv')
print('inbound_fname:', outbound_fname)

device_distribution_fname = os.path.join(save_dir, 'for_South_Carolina', f'state_FIPS_{state_FIPS:02}_monthly_device_distribution_{year}{month:02}.csv')
print('device_distribution_fname:', device_distribution_fname)

device_sampling_rate_fname = os.path.join(save_dir, 'for_South_Carolina', f'state_FIPS_{state_FIPS:02}_monthly_device_sampling_rate_{year}{month:02}.csv')
print('device_sampling_rate_fname:', device_sampling_rate_fname)

In [ ]:
# Outbound
IPF_df = pd.read_csv(IPF_results_fname, dtype={'CBG':str}).set_index('CBG')

hour_cnt = IPF_df.shape[1]  
print(f"hour cnt: {hour_cnt}")

print("IPF total outbound (billion):", IPF_df.iloc[:, :].sum().sum()/ 10**9)

IPF_df

In [ ]:
# outbound_fname
extracted_IPF_df = IPF_df[IPF_df.index.str.startswith(state_FIPS)]
extracted_IPF_df.round(0).to_csv(outbound_fname, index=True)
extracted_IPF_df

In [ ]:
# Inbound
try:
    restored_actual_CBG_hourly_visitor_df = pd.read_csv(inbound_hourly_df_fname, encoding='utf-8', dtype={'CBG':str}).set_index('CBG')
except:
    restored_actual_CBG_hourly_visitor_df = pd.read_csv(inbound_hourly_df_fname, encoding='utf-16', dtype={'CBG':str}).set_index('CBG')

restored_actual_CBG_hourly_visitor_df = restored_actual_CBG_hourly_visitor_df.iloc[:, :hour_cnt]
print("Total restored visitor count (billion):", restored_actual_CBG_hourly_visitor_df.sum().sum() / 10**9)
restored_actual_CBG_hourly_visitor_df

In [ ]:
# inbound_fname
extracted_inboud_df = restored_actual_CBG_hourly_visitor_df[restored_actual_CBG_hourly_visitor_df.index.str.startswith(state_FIPS)]
extracted_inboud_df.to_csv(inbound_fname, index=True)
extracted_inboud_df

In [ ]:
# Load the hourly map
hourly_population_map_df = pd.read_csv(CBG_hourly_population_fname, dtype={'CBG':str}).set_index('CBG')
hourly_population_map_df

In [ ]:
# houry_population_fname
houry_population_df = hourly_population_map_df[hourly_population_map_df.index.str.startswith(state_FIPS)]
houry_population_df.round(0).to_csv(houry_population_fname, index=True)
houry_population_df

In [ ]:
# device distribution
device_home_areas_df = pd.read_csv(device_home_areas_fname, dtype={"origin":str, "destination":str})
device_home_areas_df


In [ ]:
# device_distribution_fname
extracted_device_distribution_df = device_home_areas_df[device_home_areas_df['destination'].str.startswith(state_FIPS)]
extracted_device_distribution_df = extracted_device_distribution_df[extracted_device_distribution_df['origin'].str.startswith(state_FIPS)]

extracted_device_distribution_df.round(0).to_csv(device_distribution_fname, index=False)
extracted_device_distribution_df

In [ ]:
# Load device sampling rate
print("Loading people_per_device_df from:", person_per_device_fname)
person_per_device_df = pd.read_csv(person_per_device_fname, dtype={"CBG_FIPS": str})

print("Total totalpopulation (billion):", person_per_device_df['totalpopulation'].sum() / 10**9)
person_per_device_df

In [ ]:
# device_sampling_rate_fname
extracted_person_per_device_df = person_per_device_df[person_per_device_df['CBG_FIPS'].str.startswith(state_FIPS)]
extracted_person_per_device_df.to_csv(device_sampling_rate_fname, index=False)
extracted_person_per_device_df

# Analysis the device count

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  

pd.set_option('display.max_columns', None)

import logging
# Create a logger
logger_name = 'all_logger'
logger = logging.getLogger(logger_name)

In [ ]:
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo

# Dell desktop
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # 2018 desktop
save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell'   
# data_dir = r'D:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'  
data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'    
ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
home_panel_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\home_panel_summary_2019_2023_restated_20240119.csv.gz"  ## large change!!!

# home_panel_fname = r"E:\OneDrive_PSU\OneDrive - The`1

# data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Dell
# ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Lenovo
# ACS_file = r"F:\Research\Wild_fire\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv"
CBG_2019_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"

# # 2018 desktop
# # save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # 2018 desktop
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2023_home_panel'  # 2018 desktop
# # data_dir = r'D:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'  
# data_dir = r'D:\SafeGraph\Advan_2024_API\Neighborhood_Patterns'    
# ACS_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
# home_panel_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\home_panel_summary_2019_2023_restated_20240119.csv.gz"  ## large change!!!
# # https://community.deweydata.io/t/residing-device-count-in-2023-07-and-2023-08-surged-abnomorally/26675/15
# CBG_2019_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"


save_file = False

year = '2022'
month = 9

os.makedirs(save_dir, exist_ok=True)


month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
no_dict_colum_fname = os.path.join(save_dir, f"NP_dict_column_{year}_{month:02}.csv")
device_home_areas_fname = os.path.join(save_dir, f"NP_split_device_home_areas_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_CBG_fname     = os.path.join(save_dir, f"adjusted_dwelling_time_stop_CBG_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_natinol_fname = os.path.join(save_dir, f"adjusted_dwelling_time_stop_natinol_{year}_{month:02}.csv")
person_per_device_fname = os.path.join(save_dir, f"person_per_device_{year}_{month:02}.csv")
restored_CBG_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_CBG_{year}_{month:02}.csv")
restored_target_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_target_{year}_{month:02}.csv")
CBG_device_stop_destination_distribution_fname  = os.path.join(save_dir, f"CBG_device_stop_destination_distribution_{year}_{month:02}.csv")
CBG_device_stop_origin_distribution_fname = os.path.join(save_dir, f"CBG_device_stop_origin_distribution_{year}_{month:02}.csv")
IPF_results_fname = os.path.join(save_dir, f"IPF_origin_outgoer_hourly_{year}{month:02}.csv")
CBG_hourly_population_fname = os.path.join(save_dir, f"CBG_population_hourly_{year}{month:02}.csv")

CBG_origin_outbound_distribution_sampling_rate_fname = os.path.join(save_dir, f"CBG_origin_outbound_distribution_sampling_rate_{year}{month:02}.csv")
CBG_destination_inbound_per_device_fname = os.path.join(save_dir, f"CBG_destination_inbound_per_device_{year}{month:02}.csv")

CBG_destination_inbound_fname = os.path.join(save_dir, f"CBG_destination_inbound_{year}{month:02}.csv")
inbound_hourly_df_fname = os.path.join(save_dir, f"inbound_hourly_{year}{month:02}.csv")
CBG_agregated_inbound_fname =  os.path.join(save_dir, f"CBG_agregated_inbound_{year}{month:02}.csv")
hourly_agregated_inbound_fname =  os.path.join(save_dir, f"hourly_agregated_inbound_{year}{month:02}.csv")

 
# month_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01'
# CBG_list_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\ACS_2019_CBG_list.csv"

# sqlite_fname = r'F:\Research\Wild_fire\Neighborhood_202306.db'




print("Month directory:", month_dir)
print("Save_dir directory:", save_dir)
print("ACS_file:", ACS_file)
print("home_panel file name:", home_panel_fname)

print("No dict colum file name:", no_dict_colum_fname)
print("Device home areas file name:", device_home_areas_fname)
print("Device person_per_device file name:", person_per_device_fname)


print("Device adjusted_dwelling_time_stop_CBG_fname:", adjusted_dwelling_time_stop_CBG_fname)
print("Device adjusted_dwelling_time_stop_natinol_fname:", adjusted_dwelling_time_stop_natinol_fname)

print("Restored_hourly_vistor_count_CBG file name:   ", restored_CBG_hourly_vistor_count_fname)
print("Restored_hourly_vistor_count_target file name:", restored_target_hourly_vistor_count_fname)

print("CBG_device_stop_destination_distribution file name:   ", CBG_device_stop_destination_distribution_fname)
print("CBG_device_stop_origin_distribution file name:", CBG_device_stop_origin_distribution_fname)

print("IPF_results file name:", IPF_results_fname)
print("CBG hourly population file name:", CBG_hourly_population_fname)

print("CBG_origin_outbound_distribution_sampling_rate file name:", CBG_origin_outbound_distribution_sampling_rate_fname)


In [ ]:
CBG_device_stop_destination_distribution_df = pd.read_csv(CBG_device_stop_destination_distribution_fname)
CBG_device_stop_origin_distribution_df = pd.read_csv(CBG_device_stop_origin_distribution_fname)

CBG_device_stop_origin_distribution_df

In [ ]:
CBG_device_stop_destination_distribution_df

In [ ]:
CBG_device_stop_destination_distribution_df[['device', 'stop']].sum()

In [ ]:
48/7.4

In [ ]:
CBG_device_stop_origin_distribution_df[['device', 'stop']].sum()

In [ ]:
home_panel_df = pd.read_csv(home_panel_fname)
home_panel_df

In [ ]:
# f"YEAR == '{year}' and MON == '{month}'"

In [ ]:
t_home_panel_df = home_panel_df.query(f"YEAR == {year} and MON == {month} ")
t_home_panel_df

In [ ]:
t_home_panel_df['NUMBER_DEVICES_RESIDING'].sum() / 1e6, t_home_panel_df['NUMBER_DEVICES_RESIDING'].sum() / 1e6

In [ ]:
CBG_device_stop_origin_distribution_df['device'].sum() / 1e6

In [ ]:
print(1e6)

In [ ]:
NP_split_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2023_home_panel\NP_split_device_home_areas_2023_06.csv"
NP_split_df = pd.read_csv(NP_split_fname, dtype={"orgin": str, "destination":str})
NP_split_df

In [ ]:
NP_split_df[['device', 'stop']].sum()

In [ ]:
intra_NP_split_df = NP_split_df.query(" origin == destination")
intra_NP_split_df

In [ ]:
intra_NP_split_df[['device', 'stop']].sum()

In [ ]:
inter_NP_split_df = NP_split_df.query(" origin != destination")
inter_NP_split_df

In [ ]:
inter_NP_split_df[['device', 'stop']].sum()

In [ ]:
intra_NP_split_df[['device', 'stop']].sum() / inter_NP_split_df[['device', 'stop']].sum()